## Aim: Collect and combine relevant mouse expression data from the Stemformatics data portal 

Link: https://www.stemformatics.org/workbench/download_multiple_datasets.

Stemformatics is an established gene expression data portal containing over 420 public gene expression datasets derived from microarray, RNA sequencing and single cell profiling technologies. It includes curated ‘collections’ of data relevant to cell reprogramming, as well as hematopoiesis and leukaemia.

### Samples

Set the serch field to 'species' and use 'Mus musculus' as search key. 

### Processing steps 

- Sample selection 
- Combine selected datasets based on platforms
- Combine all selected datasets

In [1]:
import pandas as pd
import numpy as np
import atlas
import handler
import requests 

/Users/monica/anaconda3/envs/stemformatics/lib/python3.7/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
# inspect the samples metadata
samples = pd.read_csv('/Users/monica/Downloads/export_metadata_samples_v7.2.4.tsv', sep='\t', index_col=2)
samples.head()

,ds_id,dataset_name,replicate_group_id,organism,sample_type,generic_sample_type,final_cell_type,parental_cell_type,sex,labelling
chip_id,,,,,,,,,,
GSM565763,6146,Polo,"Skeletal muscle precursor-derived iPS, replica...",Mus musculus,skeletal muscle precursor-derived iPSC,induced pluripotent stem cell,induced pluripotent stem cell,skeletal muscle precursor,NaN,NaN
GSM565757,6146,Polo,"B cell-derived iPS, replicate 2",Mus musculus,B cell-derived iPSC,induced pluripotent stem cell,induced pluripotent stem cell,B cell,NaN,NaN
GSM565758,6146,Polo,"B cell-derived iPS, replicate 3",Mus musculus,B cell-derived iPSC,induced pluripotent stem cell,induced pluripotent stem cell,B cell,NaN,NaN
GSM565756,6146,Polo,"B cell-derived iPS, replicate 1",Mus musculus,B cell-derived iPSC,induced pluripotent stem cell,induced pluripotent stem cell,B cell,NaN,NaN
GSM565760,6146,Polo,"Tail tip fibroblast-derived iPS, replicate 2",Mus musculus,Tail tip fibroblast-derived iPSC,induced pluripotent stem cell,induced pluripotent stem cell,fibroblast,NaN,NaN


Many of the samples are not healthy blood cells (e.g. iPSCs, AML samples etc.). We will need to select for healthy blood cells from the metadata, and download only the selected samples. 

### Sample selction 

In [3]:
def select_samples(samples):
    '''This function takes the Stemformatics samples metadata and returns samples that are annotated to be blood cells.'''
    pos_selected_id = []
    neg_selected_id = []
    patterns_pos = ['lymp', '[Hh]aem', '[Hh]em', 'HSC','[Mm]ono', '[Bb]-*\ *cell', '[Mm]yelo', 'killer',
                    'NK', '[Mm]eg', '[Bb]aso', '[Nn]eut', '[Ee]os', '[Pp]las', '[Ee]ryt', '\W[Tt]-*\ *cell', 'DC', '[Dd]endri', 
                    'phage', 'macr']
    patterns_neg = ['iPS', 'MSC', 'AML', 'reprogram', 'MAPC', 'KO', 'endothelial', 'LPS', 'mutant', 'Dusp', 'LCMV', 'LSK', 'Chaudhury', 'BLSP',
                    'Bruttger']
    for col in samples.columns:
        l = samples[samples[col].notna()]
        for p in patterns_pos:
            pos_selected_id += l[(l[col].astype(str).str.contains(p) == True)].index.tolist()
        for n in patterns_neg: 
            neg_selected_id += l[(l[col].astype(str).str.contains(n) == True)].index.tolist()
    selected = samples.loc[samples.index.isin(set(pos_selected_id))]
    return selected.loc[~selected.index.isin(set(neg_selected_id))]

In [4]:
selected_samples = select_samples(samples)
print(selected_samples.shape)
selected_samples.head() # 324 samples are selected

(319, 10)


,ds_id,dataset_name,replicate_group_id,organism,sample_type,generic_sample_type,final_cell_type,parental_cell_type,sex,labelling
chip_id,,,,,,,,,,
GSM98876,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN
GSM98877,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN
GSM98878,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN
GSM98882,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC young animal,LT-HSC young animal,NaN,NaN,NaN,NaN
GSM98879,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN


### Combine datasets based on platforms

In [5]:
# add platform information of to samples metadata
datasets = pd.read_csv('/Users/monica/Downloads/export_metadata_datasets_v7.2.4.tsv', sep='\t', index_col=0)
selected_ds = datasets[datasets.index.isin(set(selected_samples.ds_id))]
selected_samples = pd.merge(selected_samples, selected_ds[['description', 'platform']], left_on='ds_id', right_index=True)
selected_samples.columns

Index(['ds_id', 'dataset_name', 'replicate_group_id', 'organism',
       'sample_type', 'generic_sample_type', 'final_cell_type',
       'parental_cell_type', 'sex', 'labelling', 'description', 'platform'],
      dtype='object')

In [6]:
# Inspect the distribution of platforms from which the samples data were generated 
selected_samples.groupby(['platform', 'ds_id']).size()

platform                                                        ds_id
Microarray (Affymetrix MoEx-1_0-ST V1)                          6405      3
Microarray (Affymetrix MoGene-1_0-ST V1)                        6310     18
                                                                6313      4
                                                                6455      3
                                                                6831     86
                                                                7131      8
                                                                7269      9
Microarray (Affymetrix Mouse430_2 (GPL1261 and A-AFFY-45))      6498      6
                                                                6658      8
                                                                6659     27
                                                                6756     11
Microarray (Affymetrix Mouse430_2)                              6087      9
                  

In [7]:
# Group selected ds_ids based on platforms
# Note that latforms with not enough dataset representation and small sample size, including Illumina Ref-6, GPL81, MoGene2, 
# and MoEx1 are excluded. 

RNAseq_id             = [7224, 7267, 6655, 6767]
Illu_MouseWG6_id      = [6637, 7291]
Affy_Mouse430_id      = [6498, 6658, 6659, 6756, 6988, 6087, 6108, 6300]
Affy_MoGene1_id       = [6264, 6310, 6313, 6455, 6831, 7131]

In [8]:
def replace_probes(df, probe_mapping):
    '''
    Input: A microarray expression dataframe and and a probe mapping table.
    Output: The expression dataframe with index changed from probe ids to ensembl gene ids according to the supplied mapping table.
    '''
    probe_mapping.columns = ['ensembl']
    return pd.merge(df, probe_mapping, how='inner', left_index=True, right_index=True).set_index('ensembl')

def find_common_genes(dfs):
    '''
    Input: A list of expression dataframes.
    Output: A list of common genes that are appeared in all dataframes.
    '''
    common_genes = dfs[0].index
    for df in dfs:
        common_genes = common_genes.intersection(df.index)       
    return common_genes

def merge_columns(dfs, common_genes):
    '''
    Input: A list of expression datasts and a list of common genes.
    Output: A combined dataframe of all supplied datasets and keep only the common genes. 
    '''
    matrix = []
    for df in dfs:
        df = df.loc[df.index.isin(common_genes)] # Filter by common genes probed in all datasets 
        m = df.groupby(df.index).first()          # Resolve rows with duplicate indexes by grouping together with the mean value 
        matrix.append(m)
    return pd.concat(matrix, axis=1)

def merge_by_platform(id_list, probe_mapping, platform):
    '''
    This function takes a list of dataset ids and a probe mapping table as inputs, and will load the corresponding datasets into 
    variables formatted as dsxxxx with xxxx indicate the ds_id. If the function is supplied with a probe mapping, it will replace 
    the data index from probe ids to gene ids according to the probe mappintg table.
    '''
    
    serverURL = 'api.stemformatics.org'
    headers = {'Content-type': 'application/json'}
    load_failed = []
    dfs = []
    for ds_id in id_list:
        try:
            result = requests.get('https://%s/expression/%s/raw' % (serverURL, ds_id), headers=headers, verify=False).json()
            df = pd.DataFrame(result['data'], index=result['index'], columns=result['columns'])
            if probe_mapping is not None:
                df = replace_probes(df, probe_mapping) 
            dfs.append(df)
        except ValueError:
            load_failed.append(ds_id)
            continue 
    print('Successfully load {} datasets {}'.format(platform, [i for i in id_list if i not in load_failed]))
    return pd.concat(dfs, axis=1)

### Combine all selected datasets

In [9]:
import warnings
warnings.filterwarnings('ignore')

def combine_s4m_ds():   
    '''
    This function will combine all the selected datasets form the stemformatics portal. 
    '''
    # Load the selected s4m datasets from the data portal api and replace the probe ID with gene ID
    genes_WG6        = pd.read_csv('../data/raw/Stemformatics/probes_MouseWG6_v2.tsv', sep='\t', index_col = 0, header = None)
    genes_Mouse430_2 = pd.read_csv('../data/raw/Stemformatics/probes_Mouse430_2.tsv', sep='\t', index_col = 0, header = None)
    genes_MoGene1    = pd.read_csv('../data/raw/Stemformatics/probes_MoGene1.tsv', sep='\t', index_col = 0, header = None)
    
    
    df_RNAseq    = merge_by_platform(RNAseq_id, None, 'RNAseq')
    df_WG6       = merge_by_platform(Illu_MouseWG6_id, genes_WG6, 'Illumina WG6')
    df_Mouse430  = merge_by_platform(Affy_Mouse430_id, genes_Mouse430_2, 'Affymetrix Mouse430_2')
    df_MoGene1   = merge_by_platform(Affy_MoGene1_id, genes_MoGene1, 'Affymetrix MoGene1')
    
    df_Mouse430.columns = df_Mouse430.columns.str.strip('.CEL')
    df_MoGene1.columns = df_MoGene1.columns.str.strip('.CEL')
    
    dfs = [df_RNAseq, df_WG6, df_Mouse430, df_MoGene1]
    
    common_genes = find_common_genes(dfs)
    combined_df = merge_columns(dfs, common_genes)
    output = combined_df.loc[:,combined_df.columns.isin(selected_samples.index)]
    return atlas.rankTransform(output)

expression_s4m = combine_s4m_ds()

Successfully load RNAseq datasets [7224, 7267, 6767]
Successfully load Illumina WG6 datasets [6637]
Successfully load Affymetrix Mouse430_2 datasets [6498, 6658, 6659, 6756, 6988, 6108, 6300]
Successfully load Affymetrix MoGene1 datasets [6264, 6310, 6313, 6455, 6831, 7131]


In [10]:
print(expression_s4m.shape)
expression_s4m.head()

(18483, 217)


,ERR1308082,ERR1308083,ERR1308084,ERR1308085,ERR1308086,ERR1308087,ERR1308088,ERR1308089,ERR1308090,GSM2631976,...,GSM854333,GSM854334,GSM1023629,GSM1023630,GSM1023631,GSM1023632,GSM1023633,GSM1023634,GSM1023635,GSM1023636
ensembl,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,0.945734,0.906914,0.882892,0.952010,0.783125,0.906184,0.939945,0.909809,0.924633,0.945085,...,0.859222,0.850457,0.926148,0.929340,0.920467,0.921063,0.917113,0.920846,0.934967,0.934480
ENSMUSG00000000003,0.138479,0.163691,0.106449,0.136855,0.159254,0.151896,0.271385,0.077720,0.091787,0.176189,...,0.002056,0.002867,0.009144,0.005140,0.003571,0.010550,0.009252,0.009198,0.004112,0.005573
ENSMUSG00000000028,0.561300,0.595791,0.607017,0.494508,0.656685,0.537223,0.714116,0.648434,0.754098,0.544636,...,0.758048,0.766975,0.772331,0.837851,0.799708,0.882000,0.873776,0.864362,0.848185,0.700103
ENSMUSG00000000031,0.363821,0.300871,0.106449,0.136855,0.119137,0.151896,0.252556,0.077720,0.091787,0.363523,...,0.263891,0.178867,0.938051,0.909106,0.861440,0.777201,0.821458,0.852080,0.079965,0.116864
ENSMUSG00000000037,0.488043,0.473192,0.391089,0.495239,0.421387,0.454066,0.453931,0.406644,0.515284,0.176189,...,0.048423,0.080452,0.335308,0.483688,0.398610,0.332468,0.415030,0.358654,0.227939,0.373749


In [11]:
# Since some of the selected datasets failed to be downloaded from the stemformatics api, we will need to select the samples metadata 
# by the samples that are actually downloaded. 
samples_s4m = selected_samples[selected_samples.index.isin(expression_s4m.columns)]
print(samples_s4m.shape)
samples_s4m.head()

(217, 12)


,ds_id,dataset_name,replicate_group_id,organism,sample_type,generic_sample_type,final_cell_type,parental_cell_type,sex,labelling,description,platform
chip_id,,,,,,,,,,,,
GSM98876,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN,Loss of immune function and an increased incid...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM98877,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN,Loss of immune function and an increased incid...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM98878,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN,Loss of immune function and an increased incid...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM98882,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC young animal,LT-HSC young animal,NaN,NaN,NaN,NaN,Loss of immune function and an increased incid...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM98879,6658,Rossi,Lin- c-kit+ Sca1+Flk2-CD34- KLSflk2-CD34- old...,Mus musculus,LT-HSC Aged animal,LT-HSC Aged animal,NaN,NaN,NaN,NaN,Loss of immune function and an increased incid...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...


In [12]:
# Save the combined expression data and samples metadata
expression_s4m.to_csv('../data/interim/mouse_integrate/expression_s4m.tsv', sep='\t')
samples_s4m.to_csv('../data/interim/mouse_integrate/samples_s4m.tsv', sep='\t')

In [13]:
samples_s4m.shape

(217, 12)

In [14]:
samples_s4m[samples_s4m.dataset_name=='Beerman']

,ds_id,dataset_name,replicate_group_id,organism,sample_type,generic_sample_type,final_cell_type,parental_cell_type,sex,labelling,description,platform
chip_id,,,,,,,,,,,,
GSM854124,6659,Beerman,Old_HSC_12h_repl2,Mus musculus,Aged mouse HSC 12 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854079,6659,Beerman,Old_HSC_12h_repl1,Mus musculus,Aged mouse HSC 12 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854125,6659,Beerman,Old_HSC_24h_repl1,Mus musculus,Aged mouse HSC 24 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854126,6659,Beerman,Old_HSC_24h_repl2,Mus musculus,Aged mouse HSC 24 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854076,6659,Beerman,Old_HSC_3h_repl2,Mus musculus,Aged mouse HSC 3 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854075,6659,Beerman,Old_HSC_3h_repl1,Mus musculus,Aged mouse HSC 3 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854077,6659,Beerman,Old_HSC_6h_repl1,Mus musculus,Aged mouse HSC 6 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854078,6659,Beerman,Old_HSC_6h_repl2,Mus musculus,Aged mouse HSC 6 hr,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
GSM854068,6659,Beerman,HSC_12h_repl2,Mus musculus,HSC 12 hr mixed expansion culture,hematopoietic stem cell,NaN,NaN,NaN,NaN,Hematopoietic stem cells (HSCs) maintain homeo...,Microarray (Affymetrix Mouse430_2 (GPL1261 and...
